# Item 44: Use Plain Attributes Instead of Setter and Getter Methods

In [1]:
# Programmers coming to Python from other programming languages may naturally try to implement explicit
# getter and setter methods in ther classs
class OldResistor:
    def __init__(self, ohms):
        self._ohms = ohms

    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

In [3]:
# Using these setters and getters is simple, but not Pythonnic
r0 = OldResistor(50e3)
print('Before:', r0.get_ohms())
r0.set_ohms(10e3)
print('After', r0.get_ohms())

Before: 50000.0
After 10000.0


In [4]:
# Such methods are especially clumsy for operations like incementing in place
r0.set_ohms(r0.get_ohms() - 4e3)
assert r0.get_ohms() == 6e3

In [5]:
# In Python, however, we never need to implement explicit getter or setter methods. Instead, we should 
# always start your implementations with simple public attributes
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

r1 = Resistor(50e3)
r1.ohms = 10e3

In [6]:
# These attrubutes make operations like incrementing in place natural and clear
r1.ohms += 5e3

In [7]:
# Later, if we decide we need special behavior when an attribute is set, we can migrate to the @property
# decorator and its corresponding setter attribute
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0

    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

In [8]:
# Now assigning the voltage property will run the voltage setter method, which in turn will update the
# current attribute of the object to match
r2 = VoltageResistance(1e3)
print(f'Before: {r2.current:.2f} amps')
r2.voltage = 10
print(f'After:  {r2.current:.2f} amps')

Before: 0.00 amps
After:  0.01 amps


In [9]:
# Specifying a stter on a property also enables us to peform type checking and validation on values passed
# to the class
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)

    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms

In [10]:
# Assigning an invalid resistance to the attribute now raises an exception
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: ohms must be > 0; got 0

In [11]:
# An exception is also raised if we pass an invalid value to the constructor
BoundedResistance(-5)

ValueError: ohms must be > 0; got -5

In [12]:
# We can even use @property to make attributes from parent classes immutable
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
     
    @property
    def ohms(self):
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Ohms is immutable")
        self._ohms = ohms

In [13]:
# Trying to assign to the property after construction raises an exception
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Ohms is immutable

When we use `@property` methods to implement setters and getters, be sure that the behavior we implement is not suprising.

In [14]:
# For example, dont't set other attributes in getter property methods
class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

In [15]:
# Setting other in getter property methods leads to extremely bizarre behavior
r7 = MysteriousResistor(10)
r7.current = 0.01
print(f'Before: {r7.voltage:.2f}')
r7.ohms
print(f'After: {r7.voltage:.2f}')

Before: 0.00
After: 0.10
